In [25]:
## Importing libraries
import pandas as pd
import pyreadstat


In [26]:
### Import Labor force survey 
## 2556
LFS_56_Q1, meta_56_Q1 = pyreadstat.read_sav("D:\Onedrive-CEU\OneDrive - Central European University\CEU\Thesis\Thesis_code\Data\LFS\LFS561.SAV")

In [27]:
LFS_56_Q1

,REG,CWD,AMP,TMB,AREA,ED,VILLAGE,BLKV,ROTA_GR,SAMSET,...,EFFECT_ND,EFFECT_RD,LAY_OFF,ASSIST,Desire,Course,Ryear,Dcourse,No_Desire,yround
0,1.0,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,561.0
1,1.0,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,561.0
2,1.0,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,561.0
3,1.0,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561.0
4,1.0,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,5.0,561.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226611,5.0,96.0,13.0,3.0,2.0,9.0,7.0,54.0,3.0,B,...,NaN,NaN,NaN,NaN,1.0,31.0,0.0,NaN,NaN,561.0
226612,5.0,96.0,13.0,3.0,2.0,9.0,7.0,54.0,3.0,B,...,NaN,NaN,NaN,NaN,1.0,31.0,0.0,NaN,NaN,561.0
226613,5.0,96.0,13.0,3.0,2.0,9.0,7.0,54.0,3.0,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561.0
226614,5.0,96.0,13.0,3.0,2.0,9.0,7.0,54.0,3.0,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561.0


In [28]:
### What does each column mean
meta_56_Q1.column_names_to_labels

{'REG': 'REGION',
 'CWD': 'CHANGWAD IN REGION',
 'AMP': None,
 'TMB': None,
 'AREA': 'AREA',
 'ED': None,
 'VILLAGE': None,
 'BLKV': None,
 'ROTA_GR': 'ROTATION GROUP',
 'SAMSET': 'SAMPLE SET',
 'MONTH': 'MONTH',
 'YEAR': 'YEAR',
 'HH_NO': 'HOUSEHOLD NUMBER',
 'HH_TYPE': 'HOUSEHOLD TYPE',
 'HH_MEM': 'NUMBER OF PERSON IN HOUSEHOLD',
 'LISTING': 'LISTING OF PERSON IN HOUSEHOLD',
 'ENUM': 'ENUMERATION',
 'LINE_NO': 'HH MEMBER NO.',
 'RELA': 'RELATION TO HEAD OF HH',
 'SEX': 'SEX',
 'AGE': 'AGE',
 'MRSTAT': 'MARITAL STATUS',
 'STUDYLEV': 'ATTENDING SCHOOL LEVEL',
 'GRADE': 'GRADE COMPLETED',
 'SUBJECT': 'SUBJECT COMPLETED',
 'LINE': 'SUBJECT FOR HIGHEST EDUCATION COMPLETED',
 'WKLW': 'WORK LAST WEEK',
 'RECEIVE': 'RECEIVE INCOME',
 'PERJOB': 'HAVE PERMANENT JOB',
 'ABSENT': 'HOW LONG LEAVE',
 'LOOKWK': 'LOOKED FOR WORK',
 'METHOD': 'HOW LOOKED FOR WORK',
 'AVAIWK': 'AVAILABLE FOR WORK',
 'REUNWK': 'REASON UNAVAILABLE FOR WORK',
 'RE_NLOOK': 'REASON NOT LOOKING FOR WORK',
 'LENLKWK': 'HOW L

In [29]:
## Label of each column
meta_56_Q1.variable_to_label

{'REG': 'labels0',
 'AREA': 'labels1',
 'HH_TYPE': 'labels2',
 'RELA': 'labels3',
 'SEX': 'labels4',
 'MRSTAT': 'labels5',
 'LINE': 'labels6',
 'WKLW': 'labels7',
 'RECEIVE': 'labels8',
 'PERJOB': 'labels9',
 'ABSENT': 'labels10',
 'LOOKWK': 'labels11',
 'METHOD': 'labels12',
 'AVAIWK': 'labels13',
 'REUNWK': 'labels14',
 'RE_NLOOK': 'labels15',
 'LENLKWK': 'labels16',
 'EVERWK': 'labels17',
 'RESNUEMP': 'labels18',
 'DR_UNEM': 'labels19',
 'WKSTAT': 'labels20',
 'FIRMSIZE': 'labels21',
 'ADD_HWK': 'labels22',
 'FINDING': 'labels23',
 'RENOMORE': 'labels24',
 'TWAGE': 'labels25',
 'LASTMEM': 'labels26',
 'RAISE_WG': 'labels27',
 'SUIT_WG': 'labels28',
 'EFFECT_ST': 'labels29',
 'EFFECT_ND': 'labels30',
 'EFFECT_RD': 'labels31',
 'LAY_OFF': 'labels32',
 'ASSIST': 'labels33',
 'Desire': 'labels34',
 'Ryear': 'labels35',
 'No_Desire': 'labels36'}

In [30]:
### Value labels (What does each value in each column mean)

meta_56_Q1.value_labels

{'labels0': {1.0: 'BANGKOK AND METROPOLIS',
  2.0: 'CENTRAL',
  3.0: 'NORTH',
  4.0: 'NORTHEAST',
  5.0: 'SOUTH'},
 'labels1': {1.0: 'MUNICIPAL', 2.0: 'NON-MUNICIPAL'},
 'labels2': {1.0: 'PRIVATE HOUSEHOLD', 2.0: 'SPECIAL HOUSEHOLD'},
 'labels3': {0.0: 'MEMBER IN SPECIAL HH',
  1.0: 'HEAD OF HH',
  2.0: 'SPOUSE',
  3.0: 'UNMARRIED CHILD',
  4.0: 'MARRIED CHILD',
  5.0: 'CHILD IN-LAW',
  6.0: 'GRAND CHILD',
  7.0: 'PARENTS',
  8.0: 'RELATIVES',
  9.0: 'OTHERS'},
 'labels4': {1.0: 'MALE', 2.0: 'FEMALE'},
 'labels5': {1.0: 'SINGLE',
  2.0: 'MARRIED',
  3.0: 'WIDOW',
  4.0: 'DIVORCED',
  5.0: 'SEPERATED',
  6.0: 'MARRIED BUT UNKNOWN'},
 'labels6': {1.0: 'ACADEMIC', 2.0: 'VOCATIONAL', 3.0: 'TEACHER TRAINING'},
 'labels7': {1.0: 'WORKED', 2.0: "DIDN'T WORK"},
 'labels8': {1.0: 'RECIEVED', 2.0: "DIN'T RECIEVED"},
 'labels9': {1.0: 'HAD', 2.0: "DIDN'T HAVE"},
 'labels10': {1.0: 'LE 2 MONTH', 2.0: 'GT 2 MONTH'},
 'labels11': {1.0: 'LOOKED 7 DAYS',
  2.0: 'LOOKED 8-30 DAYS',
  3.0: "DIDN'T LOOK"

In [31]:
## Import new province code.
New_province = pd.read_csv("D:\Onedrive-CEU\OneDrive - Central European University\CEU\Thesis\Thesis_code\Data\Province_code.csv")

## Merge with LFS.
LFS_56_Q1 = pd.merge(LFS_56_Q1, New_province[["REG_NEW", "CWD"]], how = 'left', left_on = 'CWD', right_on = 'CWD')

LFS_56_Q1


,REG,CWD,AMP,TMB,AREA,ED,VILLAGE,BLKV,ROTA_GR,SAMSET,...,EFFECT_RD,LAY_OFF,ASSIST,Desire,Course,Ryear,Dcourse,No_Desire,yround,REG_NEW
0,1.0,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,561.0,1
1,1.0,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,561.0,1
2,1.0,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,561.0,1
3,1.0,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561.0,1
4,1.0,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,5.0,561.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226611,5.0,96.0,13.0,3.0,2.0,9.0,7.0,54.0,3.0,B,...,NaN,NaN,NaN,1.0,31.0,0.0,NaN,NaN,561.0,6
226612,5.0,96.0,13.0,3.0,2.0,9.0,7.0,54.0,3.0,B,...,NaN,NaN,NaN,1.0,31.0,0.0,NaN,NaN,561.0,6
226613,5.0,96.0,13.0,3.0,2.0,9.0,7.0,54.0,3.0,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561.0,6
226614,5.0,96.0,13.0,3.0,2.0,9.0,7.0,54.0,3.0,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561.0,6


In [32]:
## move REG_NEW to the first column, drop REG.
cols = LFS_56_Q1.columns.tolist()
cols = cols[-1:] + cols[:-1]
LFS_56_Q1 = LFS_56_Q1[cols]

## drop REG.
LFS_56_Q1 = LFS_56_Q1.drop(columns = ["REG"])

In [33]:
LFS_56_Q1

,REG_NEW,CWD,AMP,TMB,AREA,ED,VILLAGE,BLKV,ROTA_GR,SAMSET,...,EFFECT_ND,EFFECT_RD,LAY_OFF,ASSIST,Desire,Course,Ryear,Dcourse,No_Desire,yround
0,1,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,561.0
1,1,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,561.0
2,1,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,561.0
3,1,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561.0
4,1,10.0,1.0,2.0,1.0,17.0,0.0,151.0,2.0,A,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,5.0,561.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226611,6,96.0,13.0,3.0,2.0,9.0,7.0,54.0,3.0,B,...,NaN,NaN,NaN,NaN,1.0,31.0,0.0,NaN,NaN,561.0
226612,6,96.0,13.0,3.0,2.0,9.0,7.0,54.0,3.0,B,...,NaN,NaN,NaN,NaN,1.0,31.0,0.0,NaN,NaN,561.0
226613,6,96.0,13.0,3.0,2.0,9.0,7.0,54.0,3.0,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561.0
226614,6,96.0,13.0,3.0,2.0,9.0,7.0,54.0,3.0,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561.0


###  Value labels of 'WKCODE': 'WORK STATUS RECODE', and 'EDCODE': 'EDUCATION RECODE'.
#### WKCODE

For 2528 - 2543:
- 01-08: Employed
- 09-12, 14-17: Unemployed
- **19-20, 22-26: Not in labor force** &rarr; 01-18, 21: Labor force
- 13, 18, 21: Waiting for season.

**Note**: must also exclude people age under 15 years old manually.

For 2544 - 2550:
- 01-03: Employed
- 04-05: Unemployed
- **06-07, 09-12: Not in labor force** &rarr; 01-05, 08: Labor force
- 08: Waiting for season.
- 13: Persons under 15 years old

For 2551 - recent:
- 01-03: Employed
- 04-05: Unemployed
- **06-07, 09-13: Not in labor force** &rarr; 01-05, 08: Labor force
- 08: Waiting for season.
- 14: Persons under 15 years old

#### EDCODE 

For 2528 - 2543:
- 01-03: Less than elementary education
- 04: Elementary education
- 05, 07: Lower secondary education
- 06, 08: Upper secondary or Associate
- 09-10: Bachelor or higher
- if 11
    + if *OCCUP* is >= 71 & <= 74, then Upper secondary or Associate
    + if *OCCUP* is >= 75 & <= 78, then Bachelor or higher
- 12-14: Other or unknown education (excluded from the analysis)

For 2544 - 2549:
- 01-02: Less than elementary education
- 03: Elementary education
- 04: Lower secondary education
- 05-10: Upper secondary or Associate
- 11-13: Bachelor or higher
- 14-15: Other or unknown education (excluded from the analysis)


For 2550 - recent:
- 01-02: Less than elementary education
- 03: Elementary education
- 04: Lower secondary education
- 05-10: Upper secondary or Associate
- 11-13: Bachelor degree education
- 14-15: Master degree education or higher
- 16-17: Other or unknown education (excluded from the analysis)



In [49]:
### Crete functions to exclude people who are not in labor force.
## Excluding people who are not in labor force (2544 - recent)
def exclude_not_LF(LFS_df): # drop WKCODE == 06-07 or >= 09.
    x = LFS_df.drop(LFS_df[(LFS_df["WKCODE"] == 6) | (LFS_df["WKCODE"] == 7) | (LFS_df["WKCODE"] >= 9)].index)
    return x

In [50]:
## Apply the function to the data (use pipe since the input is a dataframe)
LFS_56_Q1 = LFS_56_Q1.pipe(exclude_not_LF)

In [51]:
### Define fuctions to recode EDCODE.
## Regroup EDCODE to 6 categories (2528 - 2543)
## Wait until the data has been received.

## Regroup EDCODE to 6 categories (2544 - 2549)
def EDCODE_NEW_50(df):
    if df["EDCODE"] < 2:
        return "Less than elementary"
    elif df["EDCODE"] == 3:
        return "Elementary"
    elif df["EDCODE"] == 4:
        return "Lower secondary"
    elif (df["EDCODE"] >= 5) & (df["EDCODE"] <= 10):
        return "Upper secondary or Associate"
    elif (df["EDCODE"] >= 11) & (df["EDCODE"] <= 13):
        return "Bachelor or higher"
    else: 
        return "Other/Unknown"

## Regroup EDCODE to 6 categories (2550 - recent)
def EDCODE_NEW_50(df):
    if df["EDCODE"] < 2:
        return "Less than elementary"
    elif df["EDCODE"] == 3:
        return "Elementary"
    elif df["EDCODE"] == 4:
        return "Lower secondary"
    elif (df["EDCODE"] >= 5) & (df["EDCODE"] <= 10):
        return "Upper secondary or Associate"
    elif (df["EDCODE"] >= 11) & (df["EDCODE"] <= 15):
        return "Bachelor or higher"
    else: 
        return "Other/Unknown"

In [52]:
## Apply the function to the dataframe.
LFS_56_Q1["EDCODE_NEW"] = LFS_56_Q1.apply(EDCODE_NEW_50, axis = 1)


In [53]:
## Check the total labor force (sum WT)
LFS_56_Q1["WT"].sum()

# correct!

39134120.86750001

In [39]:
### Keep only the necessary columns: REG_NEW, YEAR, AGE, EDCODE, WKCODE, OCCUP, INDUST, PERMONTH, WT
LFS_56_Q1 = LFS_56_Q1[["REG_NEW", "YEAR", "AGE", "EDCODE", "EDCODE_NEW", "WKCODE", "OCCUP", "INDUST", "PERMONTH", "WT"]]

In [40]:
LFS_56_Q1

,REG_NEW,YEAR,AGE,EDCODE,EDCODE_NEW,WKCODE,OCCUP,INDUST,PERMONTH,WT
0,1,56.0,46.0,11.0,Bachelor or higher,1.0,5221.0,47711.0,NaN,877.8005
1,1,56.0,40.0,5.0,Upper secondary or Associate,1.0,5223.0,47711.0,NaN,881.6978
2,1,56.0,22.0,3.0,Elementary,1.0,9111.0,97000.0,6000.0,545.8329
3,1,56.0,NaN,NaN,Other/Unknown,NaN,NaN,NaN,NaN,NaN
4,1,56.0,77.0,2.0,Other/Unknown,1.0,5221.0,47711.0,NaN,738.7725
...,...,...,...,...,...,...,...,...,...,...
226608,6,56.0,47.0,1.0,Less than elementary,1.0,6112.0,1261.0,NaN,378.8351
226609,6,56.0,40.0,3.0,Elementary,1.0,5120.0,85101.0,4500.0,421.1832
226610,6,56.0,21.0,3.0,Elementary,1.0,6112.0,1261.0,NaN,711.6193
226611,6,56.0,18.0,3.0,Elementary,1.0,6112.0,1261.0,NaN,677.1118


## Examine employment in the three broad sectors: agriculture, industry, and services.

From 2528 - 2543, the standard is ISIC Rev (4 digits)1:
- Agriculture: 01-04
- Industry: 11-52
- Services: 61-90

From 2544 - 2553, the standard is ISIC Rev 3. (4 digits):
- Agriculture: 01-05
- Industry: 10-45
- Services: 50-99

From 2554 - recent years, the standard is TSIC 2009 (5 digits):
- Agriculture: 01-03
- Industry: 05-43
- Services: 45-99

## Examine education level. 

In [41]:
### Tabulate the data. Get the number of people in each education level (7 categories), in each region (6 regions).
## Tabulate the data.
LFS_56_Q1_EDU_REG_TOTAL = LFS_56_Q1.groupby(["REG_NEW", "EDCODE_NEW"]).agg({"WT": "sum"}).reset_index() ## Don't sort by the alphabetical order.

## Exclude the "Other/Unknown" category.
LFS_56_Q1_EDU_REG_TOTAL = LFS_56_Q1_EDU_REG_TOTAL[LFS_56_Q1_EDU_REG_TOTAL["EDCODE_NEW"] != "Other/Unknown"].reset_index(drop=True) ## drop previous index (otherwise it becomes a new column)

## Create a new column for the percentage of each education level in each region.
LFS_56_Q1_EDU_REG_TOTAL["Percentage"] = LFS_56_Q1_EDU_REG_TOTAL.groupby("REG_NEW")["WT"].apply(lambda x: x/x.sum()*100).reset_index().WT



In [42]:
## Compute the total number of people in each education level at the national level
LFS_56_Q1_EDU_NAT_TOTAL = LFS_56_Q1_EDU_REG_TOTAL.groupby(["EDCODE_NEW"]).agg({"WT": "sum"}).reset_index()

## Create column REG_NEW for the national level. Set it to 0. 
LFS_56_Q1_EDU_NAT_TOTAL["REG_NEW"] = 0

# Make REG_NEW the first column.
cols = LFS_56_Q1_EDU_NAT_TOTAL.columns.tolist()
cols = cols[-1:] + cols[:-1]
LFS_56_Q1_EDU_NAT_TOTAL = LFS_56_Q1_EDU_NAT_TOTAL[cols]

## Create a new column for the percentage of each education level
LFS_56_Q1_EDU_NAT_TOTAL["Percentage"] = LFS_56_Q1_EDU_NAT_TOTAL["WT"]/LFS_56_Q1_EDU_NAT_TOTAL["WT"].sum()*100



In [43]:

## Concatenate the national level data to the regional level data.
LFS_56_Q1_EDU_REG_TOTAL = pd.concat([LFS_56_Q1_EDU_NAT_TOTAL, LFS_56_Q1_EDU_REG_TOTAL], axis = 0).reset_index(drop=True) ## drop previous index entirely.


In [44]:

## remove the dataframes that are no longer needed.
del LFS_56_Q1_EDU_NAT_TOTAL

In [45]:
### Sort the table by EDCODE_NEW.
custom_dict_edu = {'Less than elementary': 0, 'Elementary': 1, 'Lower secondary': 2, 'Upper secondary or Associate': 3, 'Bachelor or higher': 4, 'Other/Unknown': 5}

# Convert EDCODE_NEW to an ordered categorical data type
LFS_56_Q1_EDU_REG_TOTAL['EDCODE_NEW'] = pd.Categorical(LFS_56_Q1_EDU_REG_TOTAL['EDCODE_NEW'], categories=custom_dict_edu.keys(), ordered=True)

# Sort the table by EDCODE_NEW and REG_NEW
LFS_56_Q1_EDU_REG_TOTAL.sort_values(by=['REG_NEW', 'EDCODE_NEW'], inplace=True, ignore_index = True)


In [46]:
LFS_56_Q1_EDU_REG_TOTAL

,REG_NEW,EDCODE_NEW,WT,Percentage
0,0,Less than elementary,2.014078e+06,6.594940
1,0,Elementary,9.231139e+06,30.226633
2,0,Lower secondary,6.463361e+06,21.163764
3,0,Upper secondary or Associate,7.670410e+06,25.116150
4,0,Bachelor or higher,5.160764e+06,16.898514
5,1,Less than elementary,1.483974e+05,3.992904
6,1,Elementary,6.648583e+05,17.889226
7,1,Lower secondary,5.715394e+05,15.378312
8,1,Upper secondary or Associate,9.269860e+05,24.942251
9,1,Bachelor or higher,1.404748e+06,37.797307


In [47]:
## Focus on young labor force (15-24) which is those who just entered the labor market.
LFS_56_Q1_YOUNG = LFS_56_Q1[LFS_56_Q1["AGE"] <= 24]

In [48]:
#### Repeat the same process of tabulating the data. Focus on education level.
## Tabulate the data.
LFS_56_Q1_YOUNG_EDU_REG_TOTAL = LFS_56_Q1_YOUNG.groupby(["REG_NEW", "EDCODE_NEW"]).agg({"WT": "sum"}).reset_index() ## Don't sort by the alphabetical order.

## Exclude the "Other/Unknown" category.
LFS_56_Q1_YOUNG_EDU_REG_TOTAL = LFS_56_Q1_YOUNG_EDU_REG_TOTAL[LFS_56_Q1_YOUNG_EDU_REG_TOTAL["EDCODE_NEW"] != "Other/Unknown"].reset_index(drop=True) ## drop previous index (otherwise it becomes a new column)

## Create a new column for the percentage of each education level in each region.
LFS_56_Q1_YOUNG_EDU_REG_TOTAL["Percentage"] = LFS_56_Q1_YOUNG_EDU_REG_TOTAL.groupby("REG_NEW")["WT"].apply(lambda x: x/x.sum()*100).reset_index().WT

## Compute the total number of people in each education level at the national level
LFS_56_Q1_YOUNG_EDU_NAT_TOTAL = LFS_56_Q1_YOUNG_EDU_REG_TOTAL.groupby(["EDCODE_NEW"]).agg({"WT": "sum"}).reset_index()

## Create column REG_NEW for the national level. Set it to 0.
LFS_56_Q1_YOUNG_EDU_NAT_TOTAL["REG_NEW"] = 0

# Make REG_NEW the first column.
cols = LFS_56_Q1_YOUNG_EDU_NAT_TOTAL.columns.tolist()

cols = cols[-1:] + cols[:-1]
LFS_56_Q1_YOUNG_EDU_NAT_TOTAL = LFS_56_Q1_YOUNG_EDU_NAT_TOTAL[cols]

## Create a new column for the percentage of each education level
LFS_56_Q1_YOUNG_EDU_NAT_TOTAL["Percentage"] = LFS_56_Q1_YOUNG_EDU_NAT_TOTAL["WT"]/LFS_56_Q1_YOUNG_EDU_NAT_TOTAL["WT"].sum()*100

## Concatenate the national level data to the regional level data.
LFS_56_Q1_YOUNG_EDU_REG_TOTAL = pd.concat([LFS_56_Q1_YOUNG_EDU_NAT_TOTAL, LFS_56_Q1_YOUNG_EDU_REG_TOTAL], axis = 0).reset_index(drop=True) ## drop previous index entirely.

## remove the dataframes that are no longer needed.
del LFS_56_Q1_YOUNG_EDU_NAT_TOTAL

### Sort the table by EDCODE_NEW.
custom_dict_edu = {'Less than elementary': 0, 'Elementary': 1, 'Lower secondary': 2, 'Upper secondary or Associate': 3, 'Bachelor or higher': 4, 'Other/Unknown': 5}

# Convert EDCODE_NEW to an ordered categorical data type
LFS_56_Q1_YOUNG_EDU_REG_TOTAL['EDCODE_NEW'] = pd.Categorical(LFS_56_Q1_YOUNG_EDU_REG_TOTAL['EDCODE_NEW'], categories=custom_dict_edu.keys(), ordered=True)

# Sort the table by EDCODE_NEW and REG_NEW
LFS_56_Q1_YOUNG_EDU_REG_TOTAL.sort_values(by=['REG_NEW', 'EDCODE_NEW'], inplace=True, ignore_index = True)

LFS_56_Q1_YOUNG_EDU_REG_TOTAL


,REG_NEW,EDCODE_NEW,WT,Percentage
0,0,Less than elementary,1.193492e+05,2.609226
1,0,Elementary,9.375864e+05,20.497614
2,0,Lower secondary,1.869713e+06,40.875866
3,0,Upper secondary or Associate,1.342664e+06,29.353472
4,0,Bachelor or higher,3.048116e+05,6.663823
5,1,Less than elementary,1.438370e+04,6.425044
6,1,Elementary,3.446754e+04,15.396278
7,1,Lower secondary,8.057663e+04,35.992711
8,1,Upper secondary or Associate,5.163813e+04,23.066194
9,1,Bachelor or higher,4.280330e+04,19.119773
